In [1]:
import http.client, urllib, base64, requests
import cognitive_face as CF 
from cognitive_face import util
import time

In [2]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

/Users/Amiros/anaconda/lib/python3.5/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed autotime.py. To use it, type:
  %load_ext autotime


In [3]:
#Emotion Analysis parameters
_url = 'https://api.projectoxford.ai/emotion/v1.0/recognize'
_key = 'ead17ccf578a40b88f2bb425d11d4a66' #Here you have to paste your primary key
_maxNumRetries = 5

#Face detection parametrs
CF.Key.set('d78211e12bba41a991382ade5d68745b') #set the key

time: 3.64 ms


In [4]:
# Image path
urlImage= ''

# Emtional Anlaysis function
def processRequest( json, data, headers, params ):

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result


headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 'url': urlImage } 
data = None
params = None

time: 70.5 ms


In [5]:
# Face detect function
def detect(image, face_id=True, landmarks=False, faceRectangle= False, attributes='gender'):
    
    url = 'detect'
    headers, data, json = util.parse_image(image)
    params = {
        'returnFaceId': face_id and 'true',
        'returnFaceLandmarks': landmarks and 'false',
        'returnFaceAttributes': attributes,
    }

    return util.request('POST', url, headers=headers, params=params, json=json,
                        data=data)

time: 9.92 ms


In [6]:
#Get all the frames from S3
#start with 355
url_list = []
for i in range(355,5920,5):
    url = "https://s3-us-west-2.amazonaws.com/debateinemotion/debate1/frame%d.jpg" % i
    url_list.append(url)

time: 5.1 ms


In [7]:
# Face Detection & Emotion Analysis - UPDATED
processed = {}
face_detection = {}
count = 355
#355

for single_image in url_list:
 
    try:
        face_detection[count] = detect(single_image)

        json = { 'url': single_image } 
        processed[count] = processRequest(json, data, headers, params)  
    except:
        pass

    try:
        if face_detection[count][0]['faceRectangle'] == processed[count][0]['faceRectangle']:
            face_detection[count][0].update(processed[count][0])
            face_detection[count][1].update(processed[count][1])
        if  face_detection[count][0]['faceRectangle'] == processed[count][1]['faceRectangle']:
            face_detection[count][0].update(processed[count][1])
            face_detection[count][1].update(processed[count][0])
    except:
        pass
    
    count += 5
    
    time.sleep(3)

KeyboardInterrupt: 

time: 56.9 s


In [9]:
face_detection

{355: [{'faceAttributes': {'gender': 'female'},
   'faceId': '76cc845f-0d14-4e2c-8df1-988b576548f1',
   'faceRectangle': {'height': 172, 'left': 520, 'top': 146, 'width': 172},
   'scores': {'anger': 0.0001672929,
    'contempt': 0.00451023877,
    'disgust': 0.00026952356,
    'fear': 7.905383e-06,
    'happiness': 0.9039006,
    'neutral': 0.09033607,
    'sadness': 0.000107829219,
    'surprise': 0.0007005392}}],
 360: [{'faceAttributes': {'gender': 'female'},
   'faceId': 'af9514d3-1716-44d4-9d84-bff40a88b2c9',
   'faceRectangle': {'height': 171, 'left': 536, 'top': 156, 'width': 171},
   'scores': {'anger': 0.000222364251,
    'contempt': 0.0308945477,
    'disgust': 0.0005188993,
    'fear': 3.26102668e-06,
    'happiness': 0.653031349,
    'neutral': 0.313853323,
    'sadness': 0.0004347275,
    'surprise': 0.001041556}}],
 365: [{'faceAttributes': {'gender': 'female'},
   'faceId': '400b7285-07b2-4121-aab8-c6a3031566a3',
   'faceRectangle': {'height': 166, 'left': 542, 'top': 1

time: 42.3 ms


In [29]:
# Save the result as Json
import json
with open('debate1_full.json', 'w') as fp:
    json.dump(face_detection, fp, sort_keys=True)

time: 252 ms
